## More Info
- Paper: https://arxiv.org/pdf/2004.00452.pdf
- Repo: https://github.com/facebookresearch/pifuhd
- Project Page: https://shunsukesaito.github.io/PIFuHD/
- 1-minute/5-minute Presentation https://www.youtube.com/embed/LWDGR5v3-3o

## Requirements
- Python 3
- PyTorch tested on 1.4.0
- json
- PIL
- skimage
- tqdm
- numpy
- cv2

## Configure input data

In [ ]:
 from google.colab import drive 
 drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
cd "/content/drive/My Drive/TCD Sem2/Augmented Reality/Project/"

/content/drive/My Drive/TCD Sem2/Augmented Reality/Project


In [ ]:
#Comment this cell after first run as it will save the desired code libraries to your Google Drive
!git clone https://github.com/facebookresearch/pifuhd

In [ ]:
#Comment this cell after first run as it will save the desired code libraries to your Google Drive
!git clone https://github.com/Daniil-Osokin/lightweight-human-pose-estimation.pytorch.git

**Uploading Images**. Currently PNG, JPEG files are supported.

In [ ]:
import os

files=[]
path = '/content/drive/My Drive/TCD Sem2/Augmented Reality/Project/pifuhd/sample_images/'
for file in os.listdir(path):
  files.append(file)
files.sort()

In [ ]:
files

['bvInside.jpg',
 'bvOutside.jpg',
 'bvOutsideU.jpg',
 'fvInside.jpg',
 'fvOutside.jpg',
 'fvOutsideU.jpg',
 'lsvInside.jpg',
 'lsvOutsideU.jpg',
 'lvOutside.jpg',
 'rsvInside.jpg',
 'rsvOutside.jpg',
 'rsvOutsideU.jpg']

## Preprocess (for cropping image)

In [ ]:
cd "lightweight-human-pose-estimation.pytorch"

/content/drive/My Drive/TCD Sem2/Augmented Reality/Project/lightweight-human-pose-estimation.pytorch


In [ ]:
#Comment this cell after first run as it will save the desired code libraries to your Google Drive
!wget https://download.01.org/opencv/openvino_training_extensions/models/human_pose_estimation/checkpoint_iter_370000.pth

In [ ]:
import torch
import cv2
import numpy as np
from models.with_mobilenet import PoseEstimationWithMobileNet
from modules.keypoints import extract_keypoints, group_keypoints
from modules.load_state import load_state
from modules.pose import Pose, track_poses
import demo

def get_rect(net, images, height_size):
    net = net.eval()

    stride = 8
    upsample_ratio = 4
    num_keypoints = Pose.num_kpts
    previous_poses = []
    delay = 33
    for image in images:
        rect_path = image.replace('.%s' % (image.split('.')[-1]), '_rect.txt')
        img = cv2.imread(image, cv2.IMREAD_COLOR)
        orig_img = img.copy()
        orig_img = img.copy()
        heatmaps, pafs, scale, pad = demo.infer_fast(net, img, height_size, stride, upsample_ratio, cpu=False)

        total_keypoints_num = 0
        all_keypoints_by_type = []
        for kpt_idx in range(num_keypoints):  # 19th for bg
            total_keypoints_num += extract_keypoints(heatmaps[:, :, kpt_idx], all_keypoints_by_type, total_keypoints_num)

        pose_entries, all_keypoints = group_keypoints(all_keypoints_by_type, pafs)
        for kpt_id in range(all_keypoints.shape[0]):
            all_keypoints[kpt_id, 0] = (all_keypoints[kpt_id, 0] * stride / upsample_ratio - pad[1]) / scale
            all_keypoints[kpt_id, 1] = (all_keypoints[kpt_id, 1] * stride / upsample_ratio - pad[0]) / scale
        current_poses = []

        rects = []
        for n in range(len(pose_entries)):
            if len(pose_entries[n]) == 0:
                continue
            pose_keypoints = np.ones((num_keypoints, 2), dtype=np.int32) * -1
            valid_keypoints = []
            for kpt_id in range(num_keypoints):
                if pose_entries[n][kpt_id] != -1.0:  # keypoint was found
                    pose_keypoints[kpt_id, 0] = int(all_keypoints[int(pose_entries[n][kpt_id]), 0])
                    pose_keypoints[kpt_id, 1] = int(all_keypoints[int(pose_entries[n][kpt_id]), 1])
                    valid_keypoints.append([pose_keypoints[kpt_id, 0], pose_keypoints[kpt_id, 1]])
            valid_keypoints = np.array(valid_keypoints)
            
            if pose_entries[n][10] != -1.0 or pose_entries[n][13] != -1.0:
              pmin = valid_keypoints.min(0)
              pmax = valid_keypoints.max(0)

              center = (0.5 * (pmax[:2] + pmin[:2])).astype(np.int)
              radius = int(0.65 * max(pmax[0]-pmin[0], pmax[1]-pmin[1]))
            elif pose_entries[n][10] == -1.0 and pose_entries[n][13] == -1.0 and pose_entries[n][8] != -1.0 and pose_entries[n][11] != -1.0:
              # if leg is missing, use pelvis to get cropping
              center = (0.5 * (pose_keypoints[8] + pose_keypoints[11])).astype(np.int)
              radius = int(1.45*np.sqrt(((center[None,:] - valid_keypoints)**2).sum(1)).max(0))
              center[1] += int(0.05*radius)
            else:
              center = np.array([img.shape[1]//2,img.shape[0]//2])
              radius = max(img.shape[1]//2,img.shape[0]//2)

            x1 = center[0] - radius
            y1 = center[1] - radius

            rects.append([x1, y1, 2*radius, 2*radius])

        np.savetxt(rect_path, np.array(rects), fmt='%d')

In [ ]:
net = PoseEstimationWithMobileNet()
checkpoint = torch.load('checkpoint_iter_370000.pth', map_location='cpu')
load_state(net, checkpoint)

for file in files:
  image_path= '/content/drive/My Drive/TCD Sem2/Augmented Reality/Project/pifuhd/sample_images/%s' % file
  print(image_path)
  get_rect(net.cuda(), [image_path], 512)

/content/drive/My Drive/TCD Sem2/Augmented Reality/Project/pifuhd/sample_images/bvInside.jpg
/content/drive/My Drive/TCD Sem2/Augmented Reality/Project/pifuhd/sample_images/bvOutside.jpg
/content/drive/My Drive/TCD Sem2/Augmented Reality/Project/pifuhd/sample_images/bvOutsideU.jpg
/content/drive/My Drive/TCD Sem2/Augmented Reality/Project/pifuhd/sample_images/fvInside.jpg
/content/drive/My Drive/TCD Sem2/Augmented Reality/Project/pifuhd/sample_images/fvOutside.jpg
/content/drive/My Drive/TCD Sem2/Augmented Reality/Project/pifuhd/sample_images/fvOutsideU.jpg
/content/drive/My Drive/TCD Sem2/Augmented Reality/Project/pifuhd/sample_images/lsvInside.jpg
/content/drive/My Drive/TCD Sem2/Augmented Reality/Project/pifuhd/sample_images/lsvOutsideU.jpg
/content/drive/My Drive/TCD Sem2/Augmented Reality/Project/pifuhd/sample_images/lvOutside.jpg
/content/drive/My Drive/TCD Sem2/Augmented Reality/Project/pifuhd/sample_images/rsvInside.jpg
/content/drive/My Drive/TCD Sem2/Augmented Reality/Project

## Download the Pretrained Model

In [ ]:
cd '/content/drive/My Drive/TCD Sem2/Augmented Reality/Project/pifuhd/'

/content/drive/My Drive/TCD Sem2/Augmented Reality/Project/pifuhd


In [ ]:
#Comment this cell after first run as it will save the desired code libraries to your Google Drive
!sh ./scripts/download_trained_model.sh

## Run PIFuHD!


In [ ]:
# Warning: all images with the corresponding rectangle files under -i will be processed. 
!python -m apps.simple_test -r 256 --use_rect -i ./sample_images/

# seems that 256 is the maximum resolution that can fit into Google Colab. 
# If you want to reconstruct a higher-resolution mesh, please try with your own machine. 

Resuming from  ./checkpoints/pifuhd.pt
test data size:  12
initialize network with normal
initialize network with normal
generate mesh (test) ...
  0% 0/12 [00:00<?, ?it/s]./results/pifuhd_final/recon/result_bvInside_256.obj
  8% 1/12 [00:06<01:14,  6.80s/it]./results/pifuhd_final/recon/result_bvOutside_256.obj
 17% 2/12 [00:12<01:04,  6.40s/it]./results/pifuhd_final/recon/result_bvOutsideU_256.obj
 25% 3/12 [00:17<00:55,  6.17s/it]./results/pifuhd_final/recon/result_fvInside_256.obj
 33% 4/12 [00:23<00:47,  6.00s/it]./results/pifuhd_final/recon/result_fvOutside_256.obj
 42% 5/12 [00:28<00:40,  5.81s/it]./results/pifuhd_final/recon/result_fvOutsideU_256.obj
 50% 6/12 [00:34<00:34,  5.75s/it]./results/pifuhd_final/recon/result_lsvInside_256.obj
 58% 7/12 [00:40<00:28,  5.80s/it]./results/pifuhd_final/recon/result_lsvOutsideU_256.obj
 67% 8/12 [00:46<00:23,  5.79s/it]./results/pifuhd_final/recon/result_lvOutside_256.obj
 75% 9/12 [00:51<00:17,  5.80s/it]./results/pifuhd_final/recon/resul